# Géométrie inverse avec Talos : atteindre des cibles par optimisation

Ce notebook présente un exemple de **géométrie inverse** avec un robot humanoïde (**Talos**). L'objectif est de trouver une configuration du robot telle que **les deux mains et les deux pieds atteignent des positions cibles** données dans l'espace.

Le problème est formulé comme une **optimisation non linéaire**, et met en évidence un point crucial :  
**le quaternion de la base flottante doit toujours être de norme 1** pour représenter une rotation valide.


In [14]:
import gepetuto

### Chargement du robot et initialisation du visualiseur

Entête classique avec Pinocchio, example robot data et meshcat

In [2]:
# %load tp2/generated/inverse_geometry_talos_import
import time
import unittest

import example_robot_data as robex
import numpy as np
from numpy.linalg import norm
from scipy.optimize import fmin_bfgs
import pinocchio as pin

from schaeffler2025.meshcat_viewer_wrapper import MeshcatVisualizer

# --- Load robot model
robot = robex.load("talos")
NQ = robot.model.nq
NV = robot.model.nv

# Open the viewer
viz = MeshcatVisualizer(robot)
viz.display(robot.q0)


In [3]:
viz.viewer.jupyter_cell()

### Définition des 4 extrémités à contrôler
Les 4 effecteurs du robot correspondent à 4 repères dans la liste *frame* du modèle.

In [4]:
# %load tp2/generated/inverse_geometry_talos_1
robot.feetIndexes = [
    robot.model.getFrameId(frameName)
    for frameName in [
        "gripper_right_fingertip_3_link",
        "gripper_left_fingertip_3_link",
        "right_sole_link",
        "left_sole_link",
    ]
]


### Ajout des cibles dans l’environnement
On définit 4 cibles à atteindre qu'on visualise dans meshcat par des sphères de couleurs.

In [5]:
# %load tp2/generated/inverse_geometry_talos_2
# We define 4 targets, one for each leg.
colors = ["red", "blue", "green", "magenta"]
for color in colors:
    viz.addSphere("world/%s" % color, 0.05, color)
    viz.addSphere("world/%s_des" % color, 0.05, color)

targets = [
    np.array([-0.7, -0.2, 1.2]),
    np.array([-0.3, 0.5, 0.8]),
    np.array([0.3, 0.1, -1.1]),
    np.array([0.9, 0.9, 0.5]),
]
# Make the targets reachable
targets = [t * 0.6 for t in targets]


Affichage des cibles dans meshcat.

In [6]:
q0 = robot.q0.copy()
pin.framesForwardKinematics(robot.model,robot.data,q0)
for i in range(4):
    p_i = robot.data.oMf[robot.feetIndexes[i]]
    viz.applyConfiguration("world/%s" % colors[i], p_i)
    viz.applyConfiguration(
        "world/%s_des" % colors[i], list(targets[i]) + [1, 0, 0, 0]
    )

### Géométrie inverse comme un problème d'optimisation

On va utiliser les fonctions d'optimisation de la librairie SciPy pour calculer une configuration du robot qui permette d'atteindre les cibles. Si besoin, une initiation à SciPy.Optim est [disponible dans un notebook](appendix-scipy_optimizer.ipynb). Les solveurs attendent en général trois arguments:
- une fonction de coût, de l'espace de recherche $x$ vers un scalaire $f: x \rightarrow f(x) \in \mathbb{R}$
- une valeur initiale du vecteur de décisions $x_0$
- optionnellement, une fonction de callback pour afficher des infos manuellements, qui prend une valeur du vecteur de décision en entrée et ne retourne rien.


#### Fonction de coût

Compléter le template suivant pour implémenter le coût:
$$ c(q) = \sum_{i=0}^{i<4}  || p_i(q) - p_i^* ||^2$$ 
avec $p_i(q)$ qu'on obtient de la liste *model.frames[feetIndexes[i]]* et $p_i^*$ qui est la translation de la cible. 

In [7]:
def cost(q):
    """
    Compute score from a configuration: sum of the 4 reaching
    tasks, one for each leg.
    """
    pin.framesForwardKinematics(robot.model,robot.data,q)
    for i in range(4):
        p_i = robot.data.oMf[robot.feetIndexes[i]].translation
        print(p_i,targets[i])
    return 0


Si besoin, voici la solution.

In [17]:
%do_not_load_snippet tp2/generated/inverse_geometry_talos_cost

#### Callback
En utilisant le même modèle, implémentez une fonction affichant la configuration candidate et les sphères.

In [16]:
%do_not_load_snippet tp2/generated/inverse_geometry_talos_callback

#### Optimisation
L'optimisateur s'appelle de cette manière.

In [10]:
# %load tp2/generated/inverse_geometry_talos_optim
qopt = fmin_bfgs(cost, robot.q0, callback=callback)


## Le problème, c'est qu'il y a un problème ...
La solution trouvée est acceptable, mais il y a un bogue caché. Il devient apparent si on écarte plus les cibles.

In [12]:
targets = [t / 0.6 for t in targets ]
qopt = fmin_bfgs(cost, robot.q0, callback=callback)
viz.display(qopt)

Que se passe-t-il ? Pourquoi ? Comment y remédier ?